# LangChain & LangGraph Assessment

This notebook demonstrates:
1. LangChain RAG with Conversation Memory
2. LangGraph workflow with conditional Summarizer node
3. Reflection on LangChain vs LangGraph

## Setup and Dependencies

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langgraph.graph import StateGraph, END
from typing import TypedDict, List
import warnings
warnings.filterwarnings('ignore')

load_dotenv()
print("✓ Dependencies loaded")

## Sample Data: The Matrix Plot

In [ ]:
documents = [
    """The Matrix is a 1999 science fiction action film written and directed by the Wachowskis. 
    The film depicts a dystopian future in which humanity is unknowingly trapped inside the Matrix, 
    a simulated reality that intelligent machines have created to distract humans while using their 
    bodies as an energy source.""",
    
    """The main character is Neo, a computer programmer and hacker whose real name is Thomas Anderson. 
    Neo is contacted by Morpheus, who reveals the truth about the Matrix. Neo is believed to be 'The One', 
    a prophesied individual who will end the war between humans and machines.""",
    
    """Other key characters include Trinity, a skilled hacker and Morpheus's second-in-command who becomes 
    Neo's love interest, and Agent Smith, a sentient program designed to eliminate threats to the Matrix. 
    The film explores themes of reality, free will, and the nature of consciousness.""",
    
    """In the climactic scenes, Neo learns to manipulate the Matrix and gains superhuman abilities. 
    He can dodge bullets, fly, and even perceive the underlying code of the Matrix. The film ends with 
    Neo promising to show people a world without boundaries, where anything is possible."""
]

print(f"✓ Loaded {len(documents)} documents about The Matrix")

## Part 1: LangChain RAG with Conversation Memory

This implementation adds conversation memory so the system can answer follow-up questions.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

splits = text_splitter.create_documents(documents)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(splits, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

print(f"✓ Created vector store with {len(splits)} chunks")

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

conversational_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    verbose=False
)

print("✓ Conversational RAG chain created with memory")

### Test Conversation Memory

In [ ]:
print("Q1: Tell me about the plot of The Matrix.\n")
result1 = conversational_chain({"question": "Tell me about the plot of The Matrix."})
print(f"A1: {result1['answer']}\n")
print("-" * 80)

In [ ]:
print("\nQ2: Who is the main character?\n")
result2 = conversational_chain({"question": "Who is the main character?"})
print(f"A2: {result2['answer']}\n")
print("-" * 80)

In [ ]:
print("\nQ3: What special abilities does he develop?\n")
result3 = conversational_chain({"question": "What special abilities does he develop?"})
print(f"A3: {result3['answer']}\n")
print("\n✓ Memory working correctly - 'he' refers to Neo from previous conversation")

## Part 2: LangGraph Workflow with Conditional Summarizer

This implementation includes:
- A retriever node
- A conditional summarizer node (activates if documents are too long)
- A final answer generation node

In [ ]:
class GraphState(TypedDict):
    question: str
    documents: List[str]
    summarized_docs: str
    answer: str
    needs_summary: bool

In [ ]:
def retrieve_documents(state: GraphState) -> GraphState:
    question = state["question"]
    docs = retriever.get_relevant_documents(question)
    doc_texts = [doc.page_content for doc in docs]
    
    total_length = sum(len(doc) for doc in doc_texts)
    needs_summary = total_length > 800
    
    print(f"Retrieved {len(docs)} documents (total length: {total_length} chars)")
    print(f"Needs summarization: {needs_summary}")
    
    return {
        **state,
        "documents": doc_texts,
        "needs_summary": needs_summary
    }

def summarize_documents(state: GraphState) -> GraphState:
    documents = state["documents"]
    combined_docs = "\n\n".join(documents)
    
    summarizer_prompt = ChatPromptTemplate.from_template(
        """Summarize the following documents concisely while retaining key information:
        
        {documents}
        
        Summary:"""
    )
    
    summarizer_chain = summarizer_prompt | llm | StrOutputParser()
    summary = summarizer_chain.invoke({"documents": combined_docs})
    
    print(f"Summarized documents to {len(summary)} chars")
    
    return {
        **state,
        "summarized_docs": summary
    }

def generate_answer(state: GraphState) -> GraphState:
    question = state["question"]
    
    if state.get("needs_summary") and state.get("summarized_docs"):
        context = state["summarized_docs"]
        print("Using summarized context for answer generation")
    else:
        context = "\n\n".join(state["documents"])
        print("Using original documents for answer generation")
    
    answer_prompt = ChatPromptTemplate.from_template(
        """Answer the question based on the following context:
        
        Context: {context}
        
        Question: {question}
        
        Answer:"""
    )
    
    answer_chain = answer_prompt | llm | StrOutputParser()
    answer = answer_chain.invoke({"context": context, "question": question})
    
    return {
        **state,
        "answer": answer
    }

def should_summarize(state: GraphState) -> str:
    if state.get("needs_summary", False):
        return "summarize"
    return "generate"

print("✓ Graph nodes defined")

In [ ]:
workflow = StateGraph(GraphState)

workflow.add_node("retrieve", retrieve_documents)
workflow.add_node("summarize", summarize_documents)
workflow.add_node("generate", generate_answer)

workflow.set_entry_point("retrieve")

workflow.add_conditional_edges(
    "retrieve",
    should_summarize,
    {
        "summarize": "summarize",
        "generate": "generate"
    }
)

workflow.add_edge("summarize", "generate")
workflow.add_edge("generate", END)

graph = workflow.compile()

print("✓ LangGraph workflow compiled")

### Test LangGraph Workflow

In [ ]:
print("\n" + "=" * 80)
print("Test 1: Short query (should skip summarization)")
print("=" * 80 + "\n")

result = graph.invoke({
    "question": "Who directed The Matrix?",
    "documents": [],
    "summarized_docs": "",
    "answer": "",
    "needs_summary": False
})

print(f"\nFinal Answer: {result['answer']}")

In [ ]:
print("\n" + "=" * 80)
print("Test 2: Complex query (should trigger summarization)")
print("=" * 80 + "\n")

result = graph.invoke({
    "question": "Explain the plot of The Matrix, including the main characters and themes.",
    "documents": [],
    "summarized_docs": "",
    "answer": "",
    "needs_summary": False
})

print(f"\nFinal Answer: {result['answer']}")

## Part 3: Reflection - LangChain vs LangGraph

### When to Use LangChain vs LangGraph

**LangChain** is ideal for straightforward, linear workflows like RAG pipelines, chatbots, or simple question-answering systems. It excels at chaining components sequentially with minimal complexity. Use it when your workflow follows a predictable path and doesn't require complex branching logic.

**LangGraph** shines in complex, stateful applications requiring conditional branching, loops, or multi-agent coordination. It's better suited for workflows where decisions determine the next step, such as autonomous agents, multi-step reasoning tasks, or systems needing human-in-the-loop interventions.

**Trade-offs:**
- LangChain offers simplicity and faster development but limited control over execution flow
- LangGraph provides fine-grained control and handles complexity better but requires more setup and understanding of graph-based architectures
- LangChain has better documentation and community support; LangGraph is newer with evolving patterns

Choose based on complexity: simple chains → LangChain; complex decision trees → LangGraph.

## Summary

This notebook successfully demonstrates:

1. ✓ **LangChain with Conversation Memory**: Implemented a RAG system that maintains conversation context and correctly resolves references like 'the main character' and 'he' across multiple turns

2. ✓ **LangGraph with Conditional Summarizer**: Built a workflow with intelligent document processing that:
   - Retrieves relevant documents
   - Conditionally summarizes if total length > 800 characters
   - Generates final answer using appropriate context

3. ✓ **Reflection**: Provided analysis of when to use each framework and their respective trade-offs